In [174]:
import re
import time
import json
import requests
import pandas as pd

from pprint import pprint
from bs4 import BeautifulSoup

In [175]:
headers = {
'Host' : 'auto.ru',
'User-Agent' : 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0',
'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
'Accept-Language' : 'en-US,en;q=0.5',
'Accept-Encoding' : 'gzip, deflate, br',
'Connection' : 'keep-alive',
'Cookie' : 'autoru_gdpr=1; suid=bc8be712d638a08131a6e9e0d202d740.c5b54afb905b8c06122ab3a791bcd52b; yandexuid=2760879651631376416; my=YysBAgA%3D; counter_ga_all7=2; credit_filter_promo_popup_closed=true; bltsr=1; autoru-visits-count=2; _csrf_token=cd33b362bade8bf97053fcccb6e8a730d67cea3890bc13dd; from_lifetime=1636905126761; from=direct; autoru_sid=a%3Ag619130a126cnq19ulaaft362m5p2vb2.ce95d73e09b71591f2b431378fbd3c26%7C1636905121806.604800.JcGnv8AgAv4Xvv1e8UHmiQ.qe2Ojz2DfSD_52bJZeYXuKaARhOYKurZFIDQMePmObs; autoruuid=g619130a126cnq19ulaaft362m5p2vb2.ce95d73e09b71591f2b431378fbd3c26; X-Vertis-DC=sas; yuidlt=1; crookie=uhdn1VGOHoS1oc8l4sXp2M5D/rZ7bF0rzNm+BarwG5j6c314PffzGRMzarsbN0UdUvjdLE2fxvOarzvBJ/iU56YxYuU=; cmtchd=MTYzNjkwNTEyNDY5Mw==; deal_million_popup_page_seen=0; los=1',
'Upgrade-Insecure-Requests' : '1',
'Sec-Fetch-Dest' : 'document',
'Sec-Fetch-Mode' : 'navigate',
'Sec-Fetch-Site' : 'none',
'Sec-Fetch-User' : '?1'
}
url = 'https://auto.ru/cars/all/'
page_suffix = '?page='

In [176]:
perks = ['Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'bodyType', 'brand', 'color', 'complectation_dict', 'description', 'engineDisplacement', 
      'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 
      'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 
      'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'car_url']

In [180]:
def get_ad_info(url, headers, characteristics):
    ad_info = dict()
    ad_id = re.search('[a-z0-9]{8,12}-[a-z0-9]{6,10}', link).group(0)
    r = requests.get(url, headers=headers)
    html = BeautifulSoup(r.content, 'html.parser')
    for li in html.find_all('li', 'CardInfoRow'):
        var1, var2 = tuple(li.children)
        key = var1.string
        value = var2.string
        if key in perks[:7]:
            ad_info[key] = value

    j1 = json.loads(html.script.string)
    j2 = json.loads(html.find('script', id="initial-state").string)
    for k in set(perks).intersection(set(j1)):
        ad_info[k] = j1[k]

    for l in set(perks).intersection(j1['vehicleEngine']):
        try:
            ad_info[l] = j1['vehicleEngine'][l]
        except:
            pass

    for m in set(perks).intersection(j2['card']['state']):
        try:
            ad_info[m] = j2['card']['state'][m]
        except:
            pass

    try: 
        ad_info['equipment_dict'] = j2['card']['vehicle_info']['equipment'] 
    except: 
        pass
    
    try: 
        ad_info['complectation_dict'] = j2['card']['vehicle_info']['complectation'] 
    except: 
        pass
    
    try: 
        ad_info['model_info'] = j2['card']['vehicle_info']['model_info'] 
    except: 
        pass
    
    try: 
        ad_info['model_name'] = j2['card']['vehicle_info']['model_info']['name'] 
    except: 
        pass
    
    try: 
        ad_info['super_gen'] = j2['card']['vehicle_info']['super_gen'] 
    except: 
        pass
    
    try: 
        ad_info['vendor'] = j2['card']['vehicle_info']['vendor'] 
    except: 
        pass
    
    try: 
        ad_info['sell_id'] = j2['card']['saleId'] 
    except: 
        pass
    

    try: 
        ad_info['price'] = j1['offers']['price'] 
    except: 
        pass
    
    try: 
        ad_info['priceCurrency'] = j1['offers']['priceCurrency'] 
    except: 
        pass
    
    try: 
        ad_info['car_url'] = j1['offers']['url'] 
    except: 
        pass
    

    ad_info['parsing_unixtime'] = time.time()

    return ad_info

In [183]:
df = pd.DataFrame(columns=perks)
for page_num in range(1, 100):
    if page_num <=1:
        page_url = url
    else:
        page_url = url + page_suffix + str(page_num)
    print(f'{page_url=} and {len(df)=}')
    res = requests.get(page_url, headers=headers)
    code = BeautifulSoup(res.content, 'html.parser')
    ad_links = code.find_all("a", class_="ListingItemTitle__link")
    n = len(ad_links)
    for l in ad_links:
        print(n, end=' ')
        n -= 1
        link = l['href']
        car_info = get_ad_info(link, headers, perks)
        df = df.append(car_info, ignore_index=True,)
        time.sleep(1)

page_url='https://auto.ru/cars/all/' and len(df)=0
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=2' and len(df)=38
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=3' and len(df)=76
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=4' and len(df)=114
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=5' and len(df)=152
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=6' and len(df)=190
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://aut

38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=55' and len(df)=1824
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=56' and len(df)=1862
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=57' and len(df)=1900
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=58' and len(df)=1938
38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=59' and len(df)=1976
37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 page_url='https://auto.ru/cars/all/?page=60' and len(df)=2013
3

In [188]:
import datetime

str(int((df.parsing_unixtime.max() - df.parsing_unixtime.min())/60)) + ' минут'

'122 минут'

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2962 entries, 0 to 2961
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Владельцы             2388 non-null   object 
 1   Владение              380 non-null    object 
 2   ПТС                   2388 non-null   object 
 3   Привод                2388 non-null   object 
 4   Руль                  2388 non-null   object 
 5   Состояние             2388 non-null   object 
 6   Таможня               2388 non-null   object 
 7   bodyType              2962 non-null   object 
 8   brand                 2962 non-null   object 
 9   color                 2962 non-null   object 
 10  complectation_dict    2962 non-null   object 
 11  description           2962 non-null   object 
 12  engineDisplacement    2962 non-null   object 
 13  enginePower           2962 non-null   object 
 14  equipment_dict        2962 non-null   object 
 15  fuelType             

In [189]:
df.to_csv('project6.csv')

In [190]:
df.head()

,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,bodyType,brand,color,...,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,car_url,price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,купе,MERCEDES,чёрный,...,1.637262e+09,RUB,2020,1105117076-5afcca8a,"{'id': '22284156', 'name': 'V (W213, S213, C23...",COUPE AUTOMATIC 2.0,автоматическая,EUROPEAN,https://auto.ru/cars/new/group/mercedes/e_klas...,5360064.0
1,1 владелец,2 года и 8 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен,внедорожник 5 дв.,TOYOTA,пурпурный,...,1.637262e+09,RUB,2018,1105828980-edee6d5a,"{'id': '20692838', 'name': 'IV (CA40) Рестайли...",ALLROAD_5_DOORS AUTOMATIC 2.5,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/rav_4/11...,2295000.0
2,3 или более,NaN,Дубликат,передний,Левый,Не требует ремонта,Растаможен,седан,RENAULT,серый,...,1.637262e+09,RUB,2006,1105892143-a25eb3c3,"{'id': '6321881', 'name': 'II', 'ru_name': '2'...",SEDAN MECHANICAL 1.6,механическая,EUROPEAN,https://auto.ru/cars/used/sale/renault/megane/...,320000.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,внедорожник 5 дв.,HYUNDAI,серебристый,...,1.637262e+09,RUB,2021,1105979835-fca730a5,"{'id': '22913367', 'name': 'II', 'ru_name': '2...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,KOREAN,https://auto.ru/cars/new/group/hyundai/creta/2...,1520000.0
4,3 или более,4 года,Дубликат,передний,Правый,Не требует ремонта,Растаможен,хэтчбек 5 дв.,TOYOTA,розовый,...,1.637262e+09,RUB,2000,1106014848-0b67bbc0,"{'id': '8306666', 'name': 'I (P10)', 'ru_name'...",HATCHBACK_5_DOORS AUTOMATIC 1.0,автоматическая,JAPANESE,https://auto.ru/cars/used/sale/toyota/vitz/110...,175000.0
